In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/GAN_for_Neural_Graph/ADHD"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['ADHD-200_PhenotypicKey.pdf',
 'ADHD200_training_set_80_190x190.mat',
 'ADHD200_testing_set_20_190x190.mat',
 'ADHD_20%_Data_info_all.xlsx',
 'readme.txt',
 'ADHD_80%_Data_info_all.xlsx',
 'Data',
 'AC_Brain',
 'BrainNet',
 'compared_models',
 'Preprocessing.ipynb']

#Import Libraries

In [2]:
import pandas as pd
import os
import numpy as np
import keras
import random
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras import optimizers
from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import Sequential
from keras import backend
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Add
from keras.utils import conv_utils
from keras.utils import to_categorical
from keras.engine import Layer
from keras.engine import InputSpec
from keras.datasets.fashion_mnist import load_data
from keras.constraints import Constraint
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from matplotlib import pyplot

# Load Data

In [3]:
train_data = np.load('Data/train_data.npy')
test_data = np.load('Data/test_data.npy')
train_combine = np.load('Data/train_combine.npy')
test_combine = np.load('Data/test_combine.npy')

print(train_data.shape, test_data.shape)
print(train_combine.shape, test_combine.shape)

(758, 190, 190) (171, 190, 190)
(758, 3) (171, 3)


In [4]:
# shuffle
index = [i for i in range(train_data.shape[0])]
random.shuffle(index)
train_data = train_data[index]
train_combine = train_combine[index]

print(train_combine[:10])

[[0.         0.45305967 1.        ]
 [0.         0.34511593 1.        ]
 [2.         0.42607374 1.        ]
 [0.         0.31965032 0.        ]
 [0.         0.5290764  1.        ]
 [1.         0.45496009 1.        ]
 [2.         0.37932345 1.        ]
 [0.         0.36754086 0.        ]
 [0.         0.39870772 1.        ]
 [2.         0.50361079 1.        ]]


In [5]:
def loadDataset():
  return train_data, train_combine, test_data, test_combine

In [6]:
# create encoded_data
temp_encoded = np.concatenate((train_combine, test_combine), axis=0)
temp_onehot = to_categorical(temp_encoded[:, 0])
print(temp_onehot.shape)

encoded_data = np.zeros((929, 5))
encoded_data[:, :3] = temp_onehot
encoded_data[:, 3:] = temp_encoded[:, 1:]

print(encoded_data.shape)
print(encoded_data[:10])

(929, 3)
(929, 5)
[[1.         0.         0.         0.45305967 1.        ]
 [1.         0.         0.         0.34511593 1.        ]
 [0.         0.         1.         0.42607374 1.        ]
 [1.         0.         0.         0.31965032 0.        ]
 [1.         0.         0.         0.5290764  1.        ]
 [0.         1.         0.         0.45496009 1.        ]
 [0.         0.         1.         0.37932345 1.        ]
 [1.         0.         0.         0.36754086 0.        ]
 [1.         0.         0.         0.39870772 1.        ]
 [0.         0.         1.         0.50361079 1.        ]]


In [7]:
# load images
def load_real_samples(seed):
  # load dataset
  X_train, combine_train, X_remain, combine_remain= loadDataset()
  
  # expand to 3d, e.g. add channels
  X_train = np.expand_dims(X_train, axis=-1)
  X_remain = np.expand_dims(X_remain, axis=-1)

  X_val, X_test, combine_val, combine_test = train_test_split(X_remain, combine_remain, test_size=0.5, random_state=seed, shuffle=True)

  # seperate label, age, gender
  y_train = combine_train[:, 0]
  y_test = combine_test[:, 0]
  y_val = combine_val[:, 0]
  as_train = combine_train[:, 1:] 
  as_test = combine_test[:, 1:]  
  as_val = combine_val[:, 1:]

  print(f"Training Data, X shape: {X_train.shape}, y shape: {y_train.shape}, as shape: {as_train.shape}")
  print(f"Validation Data, X shape: {X_val.shape}, y shape: {y_val.shape}, as shape: {as_val.shape}")
  print(f"Test Data, X shape: {X_test.shape}, y shape: {y_test.shape}, as shape: {as_test.shape}")

  output = [X_train, y_train, as_train[:, 0]],[X_val, y_val, as_val[:, 0]],[X_test, y_test, as_test[:, 0]]
  print(f"Code_train shape: {as_train[:, 0].shape}")

  return output

#Hyper parameters

In [8]:
DROPOUT = 0.5
MOMENTUM = 0.9
LR = 0.001  
DECAY = 0.0005
ALPHA = 0.33
FE_CHANNEL = 64
CODE_CHANNEL = 8
MERGE_CHANNEL = 32
BATCH_SIZE = 32

'''
The number of E2E layers
'''

'\nThe number of E2E layers\n'

In [9]:
class E2E_conv(Layer):
  def __init__(self, rank,
         filters,
         kernel_size,
         strides=1,
         padding='valid',
         activation=None,
         kernel_initializer='glorot_uniform',
         kernel_regularizer=None,
         kernel_constraint=None,
         **kwargs):
    super(E2E_conv, self).__init__(**kwargs)
    self.rank = rank
    self.filters = filters
    self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank, 'kernel_size')
    self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
    self.padding = conv_utils.normalize_padding(padding)
    self.activation = activations.get(activation)
    self.kernel_initializer = initializers.get(kernel_initializer)
    self.kernel_regularizer = regularizers.get(kernel_regularizer)
    self.kernel_constraint = constraints.get(kernel_constraint)
    self.input_spec = InputSpec(ndim=self.rank + 2)

  def build(self, input_shape):
    channel_axis = -1
    if input_shape[channel_axis] is None:
      raise ValueError('The channel dimension of the inputs'
               'should be defined. Found `None`.')
    input_dim = input_shape[channel_axis]
    kernel_shape = self.kernel_size + (input_dim, self.filters)

    self.kernel = self.add_weight(shape=kernel_shape,
                    initializer=self.kernel_initializer,
                    name='kernel',
                    regularizer=self.kernel_regularizer,
                    constraint=self.kernel_constraint)
    
    # Set input spec.
    self.input_spec = InputSpec(ndim=self.rank + 2,
                   axes={channel_axis:input_dim})
    self.built = True

  def call(self, inputs):
    kernel_shape = K.get_value(self.kernel).shape
    d = kernel_shape[1]
    kernellxd = K.reshape(self.kernel[0,:], (1, kernel_shape[1], kernel_shape[2], kernel_shape[3]))  # row vector
    kerneldxl = K.reshape(self.kernel[1,:], (kernel_shape[1], 1, kernel_shape[2], kernel_shape[3]))  # column vector
    convlxd = K.conv2d(
        inputs,
        kernellxd,
        strides=self.strides,
        padding=self.padding)
    convdxl = K.conv2d(
        inputs,
        kerneldxl,
        strides=self.strides,
        padding=self.padding)
    concat1 = K.concatenate([convdxl]*d, axis=1)
    concat2 = K.concatenate([convlxd]*d, axis=2)
    return concat1 + concat2

  def compute_output_shape(self, input_shape):
    return (input_shape[0], input_shape[1], input_shape[2], self.filters)

  def get_config(self):
    config = {
        'rank': self.rank,
        'filters': self.filters,
        'kernel_size': self.kernel_size,
        'strides': self.strides,
        'padding': self.padding,
        'activation': activations.serialize(self.activation),
        'kernel_initializer': initializers.serialize(self.kernel_initializer),
        'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
        'kernel_constraint': constraints.serialize(self.kernel_constraint)
    }
    base_config = super(E2E_conv, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

#Build Model

In [10]:
class E2E_conv(Layer):
  def __init__(self, rank,
         filters,
         kernel_size,
         strides=1,
         padding='valid',
         activation=None,
         kernel_initializer='glorot_uniform',
         kernel_regularizer=None,
         kernel_constraint=None,
         **kwargs):
    super(E2E_conv, self).__init__(**kwargs)
    self.rank = rank
    self.filters = filters
    self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank, 'kernel_size')
    self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
    self.padding = conv_utils.normalize_padding(padding)
    self.activation = activations.get(activation)
    self.kernel_initializer = initializers.get(kernel_initializer)
    self.kernel_regularizer = regularizers.get(kernel_regularizer)
    self.kernel_constraint = constraints.get(kernel_constraint)
    self.input_spec = InputSpec(ndim=self.rank + 2)

  def build(self, input_shape):
    channel_axis = -1
    if input_shape[channel_axis] is None:
      raise ValueError('The channel dimension of the inputs'
               'should be defined. Found `None`.')
    input_dim = input_shape[channel_axis]
    kernel_shape = self.kernel_size + (input_dim, self.filters)

    self.kernel = self.add_weight(shape=kernel_shape,
                    initializer=self.kernel_initializer,
                    name='kernel',
                    regularizer=self.kernel_regularizer,
                    constraint=self.kernel_constraint)
    
    # Set input spec.
    self.input_spec = InputSpec(ndim=self.rank + 2,
                   axes={channel_axis:input_dim})
    self.built = True

  def call(self, inputs):
    kernel_shape = K.get_value(self.kernel).shape
    d = kernel_shape[1]
    kernellxd = K.reshape(self.kernel[0,:], (1, kernel_shape[1], kernel_shape[2], kernel_shape[3]))  # row vector
    kerneldxl = K.reshape(self.kernel[1,:], (kernel_shape[1], 1, kernel_shape[2], kernel_shape[3]))  # column vector
    convlxd = K.conv2d(
        inputs,
        kernellxd,
        strides=self.strides,
        padding=self.padding)
    convdxl = K.conv2d(
        inputs,
        kerneldxl,
        strides=self.strides,
        padding=self.padding)
    concat1 = K.concatenate([convdxl]*d, axis=1)
    concat2 = K.concatenate([convlxd]*d, axis=2)
    return concat1 + concat2

  def compute_output_shape(self, input_shape):
    return (input_shape[0], input_shape[1], input_shape[2], self.filters)

  def get_config(self):
    config = {
        'rank': self.rank,
        'filters': self.filters,
        'kernel_size': self.kernel_size,
        'strides': self.strides,
        'padding': self.padding,
        'activation': activations.serialize(self.activation),
        'kernel_initializer': initializers.serialize(self.kernel_initializer),
        'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
        'kernel_constraint': constraints.serialize(self.kernel_constraint)
    }
    base_config = super(E2E_conv, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [11]:
def define_E2E(in_shape=(190,190,1), n_classes=3):
  # Setting l2_norm regularizer
  reg = regularizers.l2(DECAY)
  # kernel initialization
  kernel_init = initializers.he_uniform()
  in_image = Input(shape=in_shape)

  # E2E layer
  fe = E2E_conv(2, 16, (2, 190), kernel_regularizer=reg)(in_image) 
  fe = LeakyReLU(alpha=ALPHA)(fe)

  fe = E2E_conv(2, 32, (2, 190), kernel_regularizer=reg)(fe) 
  fe = LeakyReLU(alpha=ALPHA)(fe)

  # E2N layer
  temp1 = Conv2D(64, (1, 190), kernel_regularizer=reg, name='row')(fe)   
  temp2 = Conv2D(64, (190, 1), kernel_regularizer=reg, name='column')(fe) 
  temp2 = Reshape((190, 1, 64))(temp2)
  fe = Add()([temp1, temp2])                    
  fe = LeakyReLU(alpha=ALPHA)(fe)

  # N2G layer
  fe = Conv2D(128, (190, 1), kernel_regularizer=reg)(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)

  # flatten feature maps
  fe = Flatten()(fe)

  fe = Dense(FE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)
  fe = Dropout(DROPOUT)(fe)

  # code input
  code_shape = (1,)  
  in_code = Input(shape=code_shape, name='in_code')

  code = Dense(CODE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(in_code)
  code = LeakyReLU(alpha=ALPHA)(code)
  code = Dropout(DROPOUT)(code)

  # concatenate image and code
  merge = Concatenate()([fe, code])

  # # concatenate image and code
  # merge = Concatenate()([fe, in_code])

  merge = Dense(MERGE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(merge)
  merge = LeakyReLU(alpha=ALPHA)(merge)
  merge = Dropout(DROPOUT)(merge)

  out = Dense(n_classes, activation='softmax', name='out')(merge)
    
  # define model
  model = Model([in_image, in_code], out)
  return model

In [12]:
BrainNetCNN = define_E2E()
BrainNetCNN.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 190, 190, 1) 0                                            
__________________________________________________________________________________________________
e2e_conv (E2E_conv)             (None, 190, 190, 16) 6080        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 190, 190, 16) 0           e2e_conv[0][0]                   
__________________________________________________________________________________________________
e2e_conv_1 (E2E_conv)           (None, 190, 190, 32) 194560      leaky_re_lu[0][0]                
______________________________________________________________________________________________

# Training

In [13]:
# load data
train_data, val_data, test_data = load_real_samples(42)
X_train, y_train, code_train = train_data
X_val, y_val, code_val = val_data
X_test, y_test, code_test = test_data

Training Data, X shape: (758, 190, 190, 1), y shape: (758,), as shape: (758, 2)
Validation Data, X shape: (85, 190, 190, 1), y shape: (85,), as shape: (85, 2)
Test Data, X shape: (86, 190, 190, 1), y shape: (86,), as shape: (86, 2)
Code_train shape: (758,)


In [32]:
path = "./BrainNet/mode2/epoch_30.h5"
batch_size = 32

opt = optimizers.SGD(momentum=MOMENTUM,nesterov=True,lr=LR)
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=0, save_best_only=True)
earlystopping = EarlyStopping(patience=10)
callbacks_list = [checkpoint,earlystopping]

# define model
model_1 = define_E2E()
model_1.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['acc'])

# train
history = model_1.fit([X_train, code_train], y_train,
              epochs=100,
              batch_size=batch_size,
              validation_data=([X_val, code_val], y_val),
              callbacks=callbacks_list,
              shuffle=True)

Epoch 1/100
24/24 [==============================] - 7s 183ms/step - loss: 1.2348 - acc: 0.4706 - val_loss: 1.2161 - val_acc: 0.5294
Epoch 2/100
24/24 [==============================] - 4s 164ms/step - loss: 1.2152 - acc: 0.5343 - val_loss: 1.2242 - val_acc: 0.5294
Epoch 3/100
24/24 [==============================] - 4s 163ms/step - loss: 1.1617 - acc: 0.6045 - val_loss: 1.2253 - val_acc: 0.5294
Epoch 4/100
24/24 [==============================] - 4s 165ms/step - loss: 1.1824 - acc: 0.6125 - val_loss: 1.2225 - val_acc: 0.5294
Epoch 5/100
24/24 [==============================] - 4s 165ms/step - loss: 1.1844 - acc: 0.6052 - val_loss: 1.2226 - val_acc: 0.5294
Epoch 6/100
24/24 [==============================] - 4s 164ms/step - loss: 1.1841 - acc: 0.5900 - val_loss: 1.2213 - val_acc: 0.5294
Epoch 7/100
24/24 [==============================] - 4s 164ms/step - loss: 1.1492 - acc: 0.6099 - val_loss: 1.2228 - val_acc: 0.5294
Epoch 8/100
24/24 [==============================] - 4s 164ms/step - 

In [33]:
test(test_data)


Validation Metrics of Discriminator:
3/3 [==============================] - 1s 38ms/step - loss: 1.2222 - acc: 0.5602
test: 58.140
three class acc: 58.140, two class acc: 61.628
three class:
              precision    recall  f1-score   support

     class 0     0.6053    0.9388    0.7360        49
     class 1     0.4000    0.1538    0.2222        26
     class 2     0.0000    0.0000    0.0000        11

    accuracy                         0.5814        86
   macro avg     0.3351    0.3642    0.3194        86
weighted avg     0.4658    0.5814    0.4865        86

AUC:  0.6104913962056819
two class:
              precision    recall  f1-score   support

     class 0     0.6053    0.9388    0.7360        49
     class 1     0.7000    0.1892    0.2979        37

    accuracy                         0.6163        86
   macro avg     0.6526    0.5640    0.5169        86
weighted avg     0.6460    0.6163    0.5475        86

specificity: 0.9387755102040817
AUC:  0.6205184776613348


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Test

In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics

def calculate_score(test, pred):
  new_pred = np.zeros((pred.shape[0], 2))
  new_pred[:, 0] = pred[:, 0]
  new_pred[:, 1] = pred[:, 1] + pred[: ,2]
  score = new_pred[:, 1]
  return score

def test(test_dataset):
  # load model
  path = "./BrainNet/mode2/epoch_30.h5"
  model_1 = define_E2E()
  opt = optimizers.SGD(momentum=MOMENTUM,nesterov=True,lr=LR)
  model_1.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['acc'])
  model_1.load_weights(path)

  X_test, labels_test, code_test = test_dataset
  num_test = X_test.shape[0]

  print(f"\nValidation Metrics of Discriminator:")
  test_metrics = model_1.evaluate([X_test, code_test], labels_test, verbose=1)
  three_c_acc = 100 * test_metrics[1]

  # two class accuracy
  temp_pred = model_1.predict([X_test, code_test])
  labels_pred = np.argmax(temp_pred, axis=1)

  correct = np.sum(labels_pred==labels_test)
  acc = correct / num_test * 100
  print('test: %.3f' % acc)

  labels_2_test, labels_2_pred = labels_test.copy(), labels_pred.copy()
  # classify 2 as 1
  labels_2_test[labels_2_test==2] = 1
  labels_2_pred[labels_2_pred==2] = 1
  # calculate the accuracy
  correct = np.sum(labels_2_test==labels_2_pred)
  two_c_acc = correct / num_test * 100
  print('three class acc: %.3f, two class acc: %.3f' % (three_c_acc, two_c_acc))
  print("="*100)

  # three class confusion metrics
  target_names = ['class 0', 'class 1', 'class 2']
  print('three class:')

  # calculate precision, recall, f1 score
  print(classification_report(labels_test, labels_pred, target_names=target_names, digits=4))

  # calculate AUC
  onehot = to_categorical(labels_test, num_classes=3)
  AUC = metrics.roc_auc_score(onehot, temp_pred, multi_class='ovr')
  print('AUC: ', AUC)
  print("="*100)

  # two class confusion metrics
  target_names = ['class 0', 'class 1']
  print('two class:')
  # calculate precision, recall, f1 score
  print(classification_report(labels_2_test, labels_2_pred, target_names=target_names, digits=4))

  # calcuate specificity
  tn, fp, fn, tp = confusion_matrix(labels_2_test, labels_2_pred).ravel()
  specificity = tn / (tn+fp)
  print('specificity:', specificity)

  # calculate AUC
  score = calculate_score(labels_2_test, temp_pred)
  AUC = metrics.roc_auc_score(labels_2_test, score)
  print('AUC: ', AUC)
  print("="*100)

In [ ]:
test(test_data)


Validation Metrics of Discriminator:
3/3 [==============================] - 1s 28ms/step - loss: 1.2545 - acc: 0.5641
test: 58.140
three class acc: 58.140, two class acc: 63.953
three class:
              precision    recall  f1-score   support

     class 0     0.6250    0.9184    0.7438        49
     class 1     0.3571    0.1923    0.2500        26
     class 2     0.0000    0.0000    0.0000        11

    accuracy                         0.5814        86
   macro avg     0.3274    0.3702    0.3313        86
weighted avg     0.4641    0.5814    0.4994        86

AUC:  0.6893292293292292
two class:
              precision    recall  f1-score   support

     class 0     0.6250    0.9184    0.7438        49
     class 1     0.7143    0.2703    0.3922        37

    accuracy                         0.6395        86
   macro avg     0.6696    0.5943    0.5680        86
weighted avg     0.6634    0.6395    0.5925        86

specificity: 0.9183673469387755
AUC:  0.6949806949806949


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
